In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## LCEL

To make it as easy as possible to create custom chains, we've implemented a "Runnable" protocol. The Runnable protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. The standard interface includes:

- stream: stream back chunks of the response
- invoke: call the chain on an input
- batch: call the chain on a list of inputs


These also have corresponding async methods:

- astream: stream back chunks of the response async
- ainvoke: call the chain on an input async
- abatch: call the chain on a list of inputs async
- astream_log: stream back intermediate steps as they happen, in addition to the final response

The input type and output type varies by component:


| Component    | Input Type                           | Output Type        |
|--------------|--------------------------------------|--------------------|
| Prompt       | Dictionary                           | PromptValue        |
| ChatModel    | Single string, list of chat messages  | ChatMessage        |
| LLM          | Single string, list of chat messages  | String             |
| OutputParser | The output of an LLM or ChatModel     | Depends on the parser |
| Retriever    | Single string                        | List of Documents  |
| Tool         | Single string or dictionary           | Depends on the tool |

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
joke_parser = PydanticOutputParser(pydantic_object=Joke)

joke_template = PromptTemplate.from_template("You are a helpful assistant that tells jokes about {joke_topic}. Tell the jokes in {joke_language} language. {format_instructions}", partial_variables={'format_instructions': joke_parser.get_format_instructions()})

chat_model = ChatOpenAI(temperature=0.5)

In [8]:
joke_chain = joke_template | chat_model | joke_parser
joke_chain.invoke({'joke_topic': 'programming', 'joke_language': 'Polish'})

Joke(setup='Dlaczego programiści nie mogą zjeść obiadu?', punchline='Bo mają za dużo pętli!')

In [9]:
joke_chain.batch([{'joke_topic': 'programming', 'joke_language': 'French'}, {'joke_topic': 'programming', 'joke_language': 'Spanish'}])

[Joke(setup='Pourquoi les programmeurs préfèrent-ils les chats ?', punchline="Parce qu'ils ont peur des dogs."),
 Joke(setup='¿Por qué los programadores prefieren el invierno?', punchline='Porque el invierno es cuando vienen los bugs')]

## Goal

We're going to create a chain of chains that:
- generates a joke with setup and punchline structure
- transforms the joke structure back into a string
- rates the joke

In [10]:
class JokeRating(BaseModel):
    joke: Joke = Field(description="joke that was rated")
    rating: int = Field(description="rating of the joke from 1 to 5")
    reason: str = Field(description="reason for the rating")
    
joke_rating_parser = PydanticOutputParser(pydantic_object=JokeRating)

joke_rating_template = PromptTemplate.from_template("Rate how funny the joke is from 1 to 5 and provide a reason for the rating. Joke: {joke}. Format instructions: {format_instructions}", partial_variables={'format_instructions': joke_rating_parser.get_format_instructions()})

rate_chain = joke_rating_template | chat_model | joke_rating_parser


In [12]:
from langchain.schema.runnable import RunnableLambda

transform_joke_chain = {'joke': joke_chain} | RunnableLambda(lambda x: f"{x['joke'].setup} {x['joke'].punchline}")
transform_joke_chain.invoke({'joke_topic': 'programming', 'joke_language': 'polish'})

'Dlaczego programista nosi okulary 3D? Bo lubi widzieć w trzech wymiarach!'

In [13]:
combined_chain = {"joke": transform_joke_chain} | rate_chain
result = combined_chain.invoke({'joke_topic': 'programming', 'joke_language': 'english'})

result.model_dump()

{'joke': {'setup': 'Why do programmers prefer dark mode?',
  'punchline': 'Because light attracts bugs.'},
 'rating': 4,
 'reason': "The joke plays on the common knowledge that bugs are attracted to light, which programmers would want to avoid. It's a clever twist on the concept and is likely to be appreciated by those in the programming field."}